<a href="https://colab.research.google.com/github/MatheusHavoc/Data-Science-Jobs-Salaries/blob/main/Data_Science_Jobs_Salaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Infelizmente Github.com não suporta gráficos em HTML, caso queira visualizar os gráficos recomendo abrir Colab pelo botão logo acima. não tem necessidade de rodar o códigos.**

#Contextualização

Conjunto de dados contendo vários cargos e salários relacionados à área de dados
Projeto Pratico para Estudo.

Por não se tratar de dados de negócios, irei descarta a necessidade de fazer **business understanding**

Então temos séries de perguntas que iremos tenta reponde com essa análise.

1.   Qual a variação de salario entre as profissões de 2020 para 2022
2.   Qual profissão teve uma melhor media salarial nos 3 anos
3.   Qual região teve a melhor media salarial
4.   Qual a corelação entre o tamanho da empresa e o salario
5.   Qual a proporção de funcionario que trabalham no mesmo local onde se localiza a empresa
6.   Qual pais mais contrata estrangeiros
7.   O tipo de trabalho influencia no salario
8.  Qual é nível de experiência da base ?
9. Preferência Trabalho remoto, presencial ou hidro por Nivel de experiência ?

# Nova seção

#Importar bibliotecas

In [292]:
import numpy as np
import pandas as pd
import plotly.express as px

#Conexão driver

In [293]:
df = pd.read_csv("/content/drive/MyDrive/arquivos análise /ds_salaries.csv")


#Entendimento e tratamento detalhado das colunas


*  trabalho_ano	O ano em que o salário foi pago.

*  nível de experiência	O nível de experiência na função durante o ano com os seguintes valores possíveis: EN Entry-level / Junior MI Mid-level / Intermediate SE Senior-level / Expert EX Executive-level / Director

*  Tipo de Emprego	O tipo de emprego para a função: PT Meio período FT Tempo integral CT Contrato FL Freelance

*  Titulo do trabalho	A função atuou durante o ano.
*  salário	O valor total do salário bruto pago.
*  salário_moeda	A moeda do salário pago como um código de moeda ISO 4217.
*  salario_em_usd	O salário em USD (taxa de câmbio dividida pela taxa média de 
USD para o respectivo ano via fxdata.foorilla.com).
*  empregado_residência	País de residência principal do funcionário durante o ano de trabalho como um código de país ISO 3166.
*  proporção_remoto	A quantidade total de trabalho feito remotamente, os valores possíveis são os seguintes: 0 Nenhum trabalho remoto (menos de 20%) 50 
Parcialmente remoto 100 Totalmente remoto (mais de 80%)
*  localização_empresa	O país da sede do empregador ou filial contratante como um código de país ISO 3166.

*  tamanho da empresa	Número médio de pessoas que trabalharam na empresa durante o ano: S menos de 50 funcionários (pequena) M 50 a 250 funcionários (média) L mais de 250 funcionários (grande)


In [294]:
df.head(5)

,Unnamed: 0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L


pandas já tem coluna index, então deletar Unnamed

In [295]:
df = df.drop(columns=["Unnamed: 0"])

vendo se tem alguma correlação entre as colunas e salários

In [296]:
corr_matrix = df.corr()
corr_salary = corr_matrix["salary"].sort_values(ascending=False)
correlated_columns = corr_matrix.loc[(corr_matrix['salary'] > 0.5) | (corr_matrix['salary'] < -0.5)].index
print(correlated_columns)

Index(['salary'], dtype='object')


In [297]:
df_salary = df.groupby(['employment_type','job_title','work_year'])['salary'].mean().reset_index()

df_salary = df_salary.rename(columns={'salary': 'salario',
                                      'employment_type': 'tipos_de_contratos_de_trabalho', 
                                      'job_title': 'cargo',
                                      'work_year': 'ano_de_trabalho'})


df_salary

,tipos_de_contratos_de_trabalho,cargo,ano_de_trabalho,salario
0,CT,Applied Machine Learning Scientist,2022,29000.00
1,CT,Business Data Analyst,2020,100000.00
2,CT,ML Engineer,2021,270000.00
3,CT,Principal Data Scientist,2021,416000.00
4,CT,Staff Data Scientist,2021,105000.00
...,...,...,...,...
107,PT,Data Engineer,2021,59000.00
108,PT,Data Engineer,2022,50000.00
109,PT,Data Scientist,2020,19000.00
110,PT,Data Scientist,2022,100000.00


O uso da mediana é recomendado quando a distribuição dos dados é considerada não normal (assimétrica) ou quando existem outliers (valores muito altos ou muito baixos em relação aos demais). A mediana representa o valor que divide a amostra em duas metades iguais, ou seja, metade dos valores está acima da mediana e metade abaixo.

Já a média é a soma de todos os valores da amostra dividido pelo número de elementos na amostra, representando a tendência central dos dados. É recomendado o uso da média quando a distribuição dos dados é considerada normal.

No caso tenho suspeito de assimetria ou outliers nos dados, é melhor usar a mediana para calcular o valor central do grupo de salários agrupados por 'employment_type', 'job_title' e 'work_year'.

In [298]:
# Agrupar os dados por categoria
df_grouped = df.groupby(['employment_type', 'job_title', 'work_year'])['salary'].mean()

# Plotar o histograma
fig = px.histogram(df_grouped, nbins=20)
fig.update_layout(template='plotly_dark')
# Exibir o gráfico
fig.show()

In [299]:
df_grouped = df.groupby(['employment_type','job_title','work_year'])['salary'].median().reset_index()

df_grouped = df_grouped.rename(columns={'salary': 'salario', 'employment_type': 'tipos_de_contratos_de_trabalho', 
                        'job_title': 'cargo', 'work_year': 'ano_de_trabalho'})

df_grouped

,tipos_de_contratos_de_trabalho,cargo,ano_de_trabalho,salario
0,CT,Applied Machine Learning Scientist,2022,29000.00
1,CT,Business Data Analyst,2020,100000.00
2,CT,ML Engineer,2021,270000.00
3,CT,Principal Data Scientist,2021,416000.00
4,CT,Staff Data Scientist,2021,105000.00
...,...,...,...,...
107,PT,Data Engineer,2021,59000.00
108,PT,Data Engineer,2022,50000.00
109,PT,Data Scientist,2020,19000.00
110,PT,Data Scientist,2022,100000.00


## comparação entre títulos de trabalho pelo salário por ano

In [300]:
df_grouped = df.groupby(['work_year', 'job_title'])['salary'].mean().reset_index()
for year, group in df_grouped.groupby('work_year'):
    fig = px.bar(group, x='job_title', y='salary', color='salary', title=f'Salário médio por título de trabalho - {year}')
    fig.update_layout(template='plotly_dark')
    fig.show()


## 1- variação de salario entre as profissões de 2020 para 2022

Calcular a variação salarial para cada profissão. Isso pode ser feito subtraindo o salário de 2020 do salário de 2022 e dividindo pelo salário de 2020. Isso lhe dará uma idéia de quais profissões tiveram a maior variação salarial.

In [301]:
pd.options.display.float_format = '{:.2f}'.format
df_grouped = df.groupby(['work_year', 'job_title'])['salary'].median()
df_pivot = df_grouped.reset_index().pivot(index='job_title', columns='work_year', values='salary')

df_pivot['difference'] = df_pivot[2022] - df_pivot[2020]
df_pivot['difference'] = df_pivot['difference'] / df_pivot[2020]

df_pivot = df_pivot.dropna(subset=['difference'])

df_pivot = df_pivot.reset_index()
df_pivot

work_year,job_title,2020,2021,2022,difference
0,AI Scientist,300000.00,33500.00,160000.00,-0.47
1,Business Data Analyst,117500.00,50000.00,745000.00,5.34
2,Computer Vision Engineer,60000.00,102000.00,67500.00,0.12
3,Data Analyst,72000.00,80000.00,105000.00,0.46
4,Data Engineer,106000.00,96575.00,120000.00,0.13
5,Data Science Manager,190200.00,207000.00,159000.00,-0.16
6,Data Scientist,60000.00,90000.00,140000.00,1.33
7,Director of Data Science,325000.00,130000.00,250000.00,-0.23
8,Lead Data Engineer,90500.00,160000.00,150000.00,0.66
9,ML Engineer,14000.00,3635000.00,20000.00,0.43


Com essas informações, é possível fazer uma análise de tendência de salários por profissão e por ano. Também é possível comparar as diferenças entre os salários de cada profissão em cada ano e calcular a variação percentual. Além disso, é possível identificar quais profissões tiveram o maior e o menor crescimento de salário ao longo dos três anos. Também é possível comparar as diferenças entre os salários de cada profissão em cada ano e calcular a variação percentual. Poderia ser criado uma tabela pivot ou gráfico para representar essas informações.

In [302]:
fig = px.bar(df_pivot, x='job_title', y='difference', color='difference', title=f'variação de salario entre as profissões de 2020 a 2022')
fig.update_layout(template='plotly_dark')
fig.show()

## 2 - Qual profissão teve uma melhor media salarial nos 3 anos

resultado: Business Data Analyst	

## 3- Qual região teve a melhor media salarial 
resultado: MY	200000.00

In [303]:
# Agrupando dados pelo campo "employee_residence"
grouped_data = df.groupby(by='employee_residence')

# Calculando a média salarial por grupo
mean_salary_by_region = grouped_data['salary_in_usd'].median()

# Transformando o resultado em um DataFrame
mean_salary_by_region = mean_salary_by_region.reset_index()

# Ordenando o DataFrame pelos maiores salários
mean_salary_by_region = mean_salary_by_region.sort_values(by='salary_in_usd', ascending=False)

# Selecionando os 5 maiores salários
top_5_salaries = mean_salary_by_region.head(5)

top_5_salaries.name = 'salary_in_usd'
top_5_salaries = top_5_salaries.reset_index()
top_5_salaries.rename(columns={'employee_residence':'região'}, inplace=True)
top_5_salaries.rename(columns={'salary_in_usd':'Salários'}, inplace=True)


top_5_salaries


,index,região,Salários
0,38,MY,200000.00
1,45,PR,160000.00
2,55,US,138475.00
3,41,NZ,125000.00
4,9,CH,122346.00


In [304]:
fig = px.bar(top_5_salaries, x='região', y='Salários', color='região', title=f'Região com a melhor média salarial')
fig.update_layout(template='plotly_dark')
fig.show()

## 4 - Qual a corelação entre o tamanho da empresa e o salario

resultado: correlação baixa 0.15%

In [305]:
# Convertendo a coluna "company_size" em dados numéricos
df['company_size'] = df['company_size'].map({'S':1,'M':2,'L':3})

# Calculando a correlação entre as colunas "salary_in_usd" e "company_size"
corr = df['salary_in_usd'].corr(df['company_size'])

# Exibindo o valor de correlação
print(corr)

0.15120545454530918


correlação fraca

## 5 - Qual a proporção de funcionario que trabalham no mesmo local onde se localiza a empresa



In [306]:
# Criando uma coluna para indicar se a localização da empresa e a residência do funcionário são iguais
df['same_location'] = df['employee_residence'] == df['company_location']

# Agrupando dados pelo campo "employee_residence"
grouped_data = df.groupby(by='employee_residence')

# Calculando  funcionários que trabalham no mesmo local onde se localiza a empresa
same_location_ratio = grouped_data['same_location'].sum()

# reset index
same_location_ratio = same_location_ratio.reset_index()

#Renomear colunas
same_location_ratio = same_location_ratio.rename(columns={"employee_residence": "localização residência"})
same_location_ratio = same_location_ratio.rename(columns={"same_location": "quantidade"})

total_employees = df['employee_residence'].count()
same_location_ratio['percentage'] = same_location_ratio['quantidade'] / total_employees * 100

# Exibindo resultados
same_location_ratio.head()

,localização residência,quantidade,percentage
0,AE,3,0.49
1,AR,0,0.00
2,AT,3,0.49
3,AU,3,0.49
4,BE,2,0.33


In [307]:
fig = px.bar(same_location_ratio, x='localização residência', y="quantidade" , color='quantidade' ,title='proporção de funcionario que trabalham no mesmo local')
fig.update_layout(template='plotly_dark')
fig.show()

## 6 - Qual pais mais contrata estrangeiros 

In [308]:
# Agrupando dados pelo campo "employee_residence"
grouped_data = df.groupby(by='employee_residence')

# Contando o número de funcionários por país
employee_count_by_country = grouped_data['salary_in_usd'].count()

# Ordenando resultados em ordem decrescente
employee_count_by_country = employee_count_by_country.sort_values(ascending=False)

# Exibindo os países com mais funcionários
employee_count_by_country = employee_count_by_country.head(10).reset_index()

employee_count_by_country = employee_count_by_country.rename(columns={"salary_in_usd": "salário"})
employee_count_by_country = employee_count_by_country.rename(columns={"employee_residence": "localização residência"})
employee_count_by_country

,localização residência,salário
0,US,332
1,GB,44
2,IN,30
3,CA,29
4,DE,25
5,FR,18
6,ES,15
7,GR,13
8,JP,7
9,PK,6


In [309]:
fig = px.bar(employee_count_by_country, x='localização residência', y='salário', color='localização residência', title=f'Pais que mais contrata estrangeiros')
fig.update_layout(template='plotly_dark')
fig.show()

## 7 -O tipo de trabalho influencia no salário 

In [310]:
# Agrupando o DataFrame pelo tipo de emprego
mean_salary_by_employment_type = df.groupby('employment_type')['salary_in_usd'].median()

# Ordenando o DataFrame pelos maiores salários
mean_salary_by_employment_type = mean_salary_by_employment_type.sort_values(ascending=False)

# Exibindo o resultado
mean_salary_by_employment_type = mean_salary_by_employment_type.reset_index()

mean_salary_by_employment_type = mean_salary_by_employment_type.rename(columns={"employment_type": "tipos de contratos de trabalhos"})
mean_salary_by_employment_type = mean_salary_by_employment_type.rename(columns={"salary_in_usd": "salário"})

mean_salary_by_employment_type

,tipos de contratos de trabalhos,salário
0,CT,105000.00
1,FT,104196.50
2,FL,40000.00
3,PT,18817.50


In [311]:
fig = px.bar(mean_salary_by_employment_type, x='tipos de contratos de trabalhos', y='salário', color='tipos de contratos de trabalhos', title=f'Tipos de contrato de trabalho X influencia no salário')
fig.update_layout(template='plotly_dark')
fig.show()

## 8 - qual é nível de experiência da base ?

In [312]:
grouped = df.groupby(['experience_level'])
df['experience_level'].replace({'EN': 'Entry Level', 'MI': 'Middle', 'SE': 'Senior', 'EX': 'Experienced'}, inplace=True)
# Criando lista de níveis de experiência
labels = list(grouped.groups.keys())

# Criando lista com o tamanho de cada grupo
values = [len(group) for group in grouped.groups.values()]

# Criando o gráfico de pizza
#fig = px.pie(values=values, labels=labels, title='Proportão de funcionários por nível de experiência')

fig = px.pie(values=values, labels=labels, color=labels, title='Proportão de funcionários por nível de experiência')

fig.update_layout(template='plotly_dark')

# Exibindo o gráfico
fig.show()

## 9 - Preferência Trabalho remoto, presencial ou hidro por Nivel de experiência ?

In [316]:

experience_order = ['Entry Level', 'Middle', 'Senior', 'Experienced']

grouped = df.groupby(['experience_level', 'remote_ratio'])['employee_residence'].count().reset_index()
grouped['proportion'] = grouped['employee_residence'] / grouped['employee_residence'].sum()

grouped['remote_ratio'].replace({0: 'None remote', 50: 'Hybrid', 100: 'Fully Remote'}, inplace=True)
grouped['experience_level'].replace({'EN': 'Entry Level', 'MI': 'Middle', 'SE': 'Senior', 'EX': 'Experienced'}, inplace=True)
grouped = grouped.sort_values('experience_level')
grouped = grouped.set_index('experience_level')


for level in experience_order:
    level_df = grouped.loc[level, :]
    fig = px.pie(level_df, values='proportion', names='remote_ratio', title='Preferência por Trabalho Remoto x Nivel de experiência: ' + level)
    fig.update_layout(template='plotly_dark')
    fig.show()
